# TODO
- [X] specify lesson plan in json format, then convert to markdown
- [ ] parallelize the lesson plan creation
- [ ] generate all activity ideas at once, so there is less repetition in the ideas (e.g., there are many small variations on the 'scavenger hunt' idea)
- [ ] try different models:
  - [ ] chat model instead of LLM
  - [ ] different providers instead of OpenAI
  - [ ] local model

In [1]:
import os
import json
import time
import pprint as pp
import pandas as pd
from pydantic import BaseModel, Field
from typing import List, Dict
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from IPython.display import display, Markdown, JSON

In [2]:
load_dotenv(override=True) # loading API key from environment

True

## Read in data with annual curriculum plan

In [3]:
GSHEET_ID = os.getenv('GSHEET_ID')

In [4]:
google_sheet_url = f"https://docs.google.com/spreadsheets/d/{GSHEET_ID}/export?format=csv&gid=0"
annual_plan_df = pd.read_csv(google_sheet_url)
annual_plan_df.rename(columns=lambda x: x.replace(" ", "_").lower(), inplace=True)
annual_plan_df

,month,theme_to_explore,language_arts,math,social_studies,science
0,August,Getting to Know One Another; Class Community,All About Me: Personal Narratives,Friendship Math: Counting and Sorting,Our Classroom Community,Exploring the school environment and local eco...
1,September,"On the Map, where we’re from",My Place on the Map: Descriptive Writing,Mapping Our Neighborhood; Measurement (units o...,My Hometown and Its History; Local History; ge...,"Water cycle, rivers; Geology, caves"
2,October,Fall; Halloween; Dia de los Muertos,Spooky Stories: Creative Writing,Place value; Estimation and graphs,"Fall celebrations (Halloween, Dia de los Muertos)",Animal science: Investigating Local Food Chains
3,November,Gratitude; Native Americans,Thankful Thoughts; Native American Storytelling,Addition and Subtraction,Native American History,Investigating Seasonal Changes and Plants
4,December,Winter Solstice and winter holidays,Holiday Traditions Around the World,Snowflake Symmetry; geometry (2-D shapes),Holidays Worldwide,Exploring Winter Animal Adaptations
5,January,"Calendar, New Year, Goals, MLK",Dreams and Goals; MLK biography,Calendar Math,New Year's Celebrations Around the World; Comm...,Climate Zones and Seasons
6,February,"Black History, Love/Valentine's Day",African American Biographies; Exploring Emotio...,"Fractions, Division; Number Patterns",African American Achievements; Civil Rights Mo...,"The human body, heart health and exercise"
7,March,Spring; Easter; Ramadan,Springtime Poetry and Descriptions,Patterns and Geometry; Geometry in Islamic Art,"Cultural Spring Celebrations (Easter, Ramadan)",Life Cycles of Plants and Animals (frogs and b...
8,April,Space; Passover,Cosmic Adventures: Space Stories,Planet Math: Solar System,Passover Origins; History of astronomy,Exploring the Solar System and Celestial Bodies
9,May,Ocean Life,Ocean Explorers: Descriptive Writing,Dive into Math: Ocean Measurements,Coastal Ecosystems,Marine Life and Habitats


## Instantiate the LLM

In [6]:
model_name = "text-davinci-003"
temperature = 0.1
model = OpenAI(model_name=model_name, temperature=temperature, max_tokens=3000)

# Create Prompt Templates

In [7]:
class ActivityIdea(BaseModel):
    name: str
    description: str

class ActivityIdeasOutput(BaseModel):
    ideas: List[ActivityIdea]
# Set up a parser + inject instructions into the prompt template.
activity_ideas_parser = PydanticOutputParser(pydantic_object=ActivityIdeasOutput)

In [8]:
activity_idea_prompt = PromptTemplate(
    template="""Answer the user query.
    {format_instructions}
    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.
    I want you to generate {num_ideas} activity ideas for the month of {month}. Each activity could also be thought of as a lesson where the kids are learning valuable skills.
    I have a theme for the month, which is {theme}.
    The academic subject is {subject} and the topic is {topic}.

    Here are some additional guidelines:
    - The microschool  stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards, so your lesson plans should be a mix of exploration and drilling the fundamentals. 
    - Try to be creative in your ideas. Do not use the idea of a scavenger hunt, because it has been overdone.
    - Try to work in phonics work and literacy development into the language art activities.
    - Try to work in arithmetic and numeracy development into the math activities.
    - Each activity idea should be able to be accomplished in 30 mins-3 hours.
    - Each activity should relate directly to skills and concepts from the academic subject of {subject}.
    - I only need you to give me the activity names and a brief description of each idea.
    - The description should only be sentence or two and should not simply repeat the theme or the topic. It should describe the nature of the activity.
    - Keep in mind these are second-graders who are not ready for advanced concepts like decimals.
    """
    ,
    input_variables=["num_ideas", "month", "theme", "subject", "topic"],
    partial_variables={"format_instructions": activity_ideas_parser.get_format_instructions()}
)

In [9]:
print(activity_ideas_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ideas": {"title": "Ideas", "type": "array", "items": {"$ref": "#/definitions/ActivityIdea"}}}, "required": ["ideas"], "definitions": {"ActivityIdea": {"title": "ActivityIdea", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "description": {"title": "Description", "type": "string"}}, "required": ["name", "description"]}}}
```


In [10]:
class LessonPlanOutput(BaseModel):
    time_estimate: str
    list_of_materials: List[str]
    procedure: List[str]
    additional_notes: List[str]
    differentiation_strategies: List[str]

# Set up a parser + inject instructions into the prompt template.
lesson_plan_parser = PydanticOutputParser(pydantic_object=LessonPlanOutput)

In [11]:
print(lesson_plan_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"time_estimate": {"title": "Time Estimate", "type": "string"}, "list_of_materials": {"title": "List Of Materials", "type": "array", "items": {"type": "string"}}, "procedure": {"title": "Procedure", "type": "array", "items": {"type": "string"}}, "additional_notes": {"title": "Additional Notes", "type": "array", "items": {"type": "string"}}, "differentiation_strategies": {"title": "Differentiation Strategies", "type": "array", "items": {"type": "string"}}}, "required": ["time_estimate", "list_of_materials", "procedure", "addition

In [12]:
lesson_plan_prompt = PromptTemplate(
    template="""Answer the user query.
    {format_instructions}
    You are a curriculum development expert and i am a teacher of second graders at a micro-school, with seven students of varying abilities.
    I want you to generate a *detailed* lesson plan.
    The name of the lesson is {activity_name}, and the short description is: {activity_description}

    Additional guidelines for the lesson plan:
    - The microschool  stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.
    - Ensure that the lesson uses and develops skills relevant to the academic subject: {subject}
    - Try to make the best estimate of the total amount of time the activity will take. Be generous because these are young kids and they take a while!
    - The additional notes should include the actual content if applicable (e.g., interview questions, puzzle clues,specific writing prompts, etc.), so the teacher does not need to do additional brainstorming to create the lesson.
    - The differentiation strategies should provide modifications for students who need more support, for students who need more challenge, and for neurodivergent students.
    """
    ,
    input_variables=["activity_name", "activity_description", "subject"],
    partial_variables={"format_instructions": lesson_plan_parser.get_format_instructions()}
)

In [13]:
print(lesson_plan_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"time_estimate": {"title": "Time Estimate", "type": "string"}, "list_of_materials": {"title": "List Of Materials", "type": "array", "items": {"type": "string"}}, "procedure": {"title": "Procedure", "type": "array", "items": {"type": "string"}}, "additional_notes": {"title": "Additional Notes", "type": "array", "items": {"type": "string"}}, "differentiation_strategies": {"title": "Differentiation Strategies", "type": "array", "items": {"type": "string"}}}, "required": ["time_estimate", "list_of_materials", "procedure", "addition

# Define Helper Functions

In [14]:
def prompt_wrapper(prompt, parser, model, max_retries=5, **kwargs):
    res ={}
    _input = prompt.format_prompt(**kwargs)
    for attempt in range(max_retries + 1):  #sometimes the parser fails inexplicably
        try:
            output = model(_input.to_string())
            parsed_output = parser.parse(output)
            break
        except Exception as e:
            if attempt < max_retries:
                # print(e)
                print(f"Attempt {attempt + 1} failed. Retrying...")
                time.sleep(2)  # Wait before retrying
            else:
                print(f"API call failed after {max_retries} attempts. Skipping.")
    return parsed_output
    

In [15]:
def make_ordered_list_markdown(lst):
    markdown_list = [f"{idx}. {val}" for idx, val in enumerate(lst, start=1)]
    return '\n'.join(markdown_list)

In [16]:
def make_list_markdown(lst):
    return "- " + "\n- ".join(lst)

In [17]:
def convert_lesson_plan_to_markdown(lesson_plan_dict):
    res = []
    res.append(f"## {lesson_plan_dict['name']}")
    res.append(lesson_plan_dict['description'])
    res.append(f"\n### Time estimate: {lesson_plan_dict['time_estimate']}")
    res.append(f"\n### Materials")
    res.append(make_list_markdown(lesson_plan_dict["list_of_materials"]))
    res.append(f"\n### Procedure")
    res.append(make_ordered_list_markdown(lesson_plan_dict['procedure']))
    res.append(f"\n### Additional notes")
    res.append(make_list_markdown(lesson_plan_dict['additional_notes']))
    res.append(f"\n### Differentiation strategies")
    res.append(make_list_markdown(lesson_plan_dict['differentiation_strategies']))
    
    return "\n".join(res)
    

## Test on single records

In [18]:
month = "September"

In [19]:
subject = 'math'

In [20]:
month_record = annual_plan_df.set_index("month").loc[month].to_dict()

In [21]:
month_record

{'theme_to_explore': 'On the Map, where we’re from',
 'language_arts': 'My Place on the Map: Descriptive Writing',
 'math': 'Mapping Our Neighborhood; Measurement (units of lenghth, weight, etc.)',
 'social_studies': 'My Hometown and Its History; Local History; geography',
 'science': 'Water cycle, rivers; Geology, caves'}

In [22]:
theme = month_record['theme_to_explore']

In [23]:
topic = month_record[subject]

In [24]:
topic

'Mapping Our Neighborhood; Measurement (units of lenghth, weight, etc.)'

In [25]:
theme

'On the Map, where we’re from'

In [26]:
num_plans = 5

In [27]:
activity_ideas_parsed = prompt_wrapper(  # Generate activity ideas for each month-subject combo
                prompt=activity_idea_prompt, 
                parser=activity_ideas_parser,
                model=model,
                num_ideas=num_plans,
                month=month,
                theme=theme,
                subject=subject, 
                topic=topic
            )

In [28]:
activity_ideas_dict = activity_ideas_parsed.dict()

In [29]:
pp.pprint(activity_ideas_dict)


{'ideas': [{'description': 'Create a map of the neighborhood using a variety '
                           'of measurement tools, such as rulers, tape '
                           'measures, and yardsticks. Have the students '
                           'measure the length and width of buildings, '
                           'streets, and other landmarks.',
            'name': 'Mapping Our Neighborhood'},
           {'description': 'Have the students measure their height, weight, '
                           'and other body parts using a variety of '
                           'measurement tools. Have them compare their '
                           'measurements to each other and discuss the '
                           'differences.',
            'name': 'Measuring Ourselves'},
           {'description': 'Have the students measure the size of objects '
                           'around them, such as furniture, toys, and other '
                           'items. Have them compare the 

In [30]:
activity_idea = activity_ideas_dict["ideas"][0]

In [31]:
activity_idea

{'name': 'Mapping Our Neighborhood',
 'description': 'Create a map of the neighborhood using a variety of measurement tools, such as rulers, tape measures, and yardsticks. Have the students measure the length and width of buildings, streets, and other landmarks.'}

In [32]:
idea_name = activity_idea["name"]
idea_description = activity_idea["description"]

In [33]:
print(f"{idea_name}: {idea_description}")

Mapping Our Neighborhood: Create a map of the neighborhood using a variety of measurement tools, such as rulers, tape measures, and yardsticks. Have the students measure the length and width of buildings, streets, and other landmarks.


In [34]:
lesson_plan_output_parsed = prompt_wrapper(
    prompt=lesson_plan_prompt, 
    parser=lesson_plan_parser, 
    model=model,
    activity_name=idea_name, 
    activity_description=idea_description, 
    subject=subject
)

In [35]:
lesson_plan_output_parsed.time_estimate

'2 hours'

In [36]:
lp_parsed_dict = lesson_plan_output_parsed.dict()

In [37]:
full_lesson_plan_dict = {k: v for k, v in activity_idea.items()}
full_lesson_plan_dict.update(lesson_plan_output_parsed.dict())

In [38]:
full_lesson_plan_dict

{'name': 'Mapping Our Neighborhood',
 'description': 'Create a map of the neighborhood using a variety of measurement tools, such as rulers, tape measures, and yardsticks. Have the students measure the length and width of buildings, streets, and other landmarks.',
 'time_estimate': '2 hours',
 'list_of_materials': ['Rulers', 'Tape measures', 'Yardsticks'],
 'procedure': ['Have the students measure the length and width of buildings, streets, and other landmarks.',
  'Have the students create a map of the neighborhood using the measurements they have taken.'],
 'additional_notes': ['Encourage the students to think critically about the measurements they are taking and how they can use them to create an accurate map.',
  'Provide examples of maps and discuss the importance of accuracy in mapping.'],
 'differentiation_strategies': ['For students who need more support: Provide additional guidance and support when measuring and creating the map.',
  'For students who need more challenge: Enco

In [39]:
lesson_plan_markdown = convert_lesson_plan_to_markdown(full_lesson_plan_dict)

In [40]:
display(Markdown(lesson_plan_markdown))

## Mapping Our Neighborhood
Create a map of the neighborhood using a variety of measurement tools, such as rulers, tape measures, and yardsticks. Have the students measure the length and width of buildings, streets, and other landmarks.

### Time estimate: 2 hours

### Materials
- Rulers
- Tape measures
- Yardsticks

### Procedure
1. Have the students measure the length and width of buildings, streets, and other landmarks.
2. Have the students create a map of the neighborhood using the measurements they have taken.

### Additional notes
- Encourage the students to think critically about the measurements they are taking and how they can use them to create an accurate map.
- Provide examples of maps and discuss the importance of accuracy in mapping.

### Differentiation strategies
- For students who need more support: Provide additional guidance and support when measuring and creating the map.
- For students who need more challenge: Encourage the students to think of creative ways to measure and create the map.
- For neurodivergent students: Provide additional breaks and allow for more time to complete the activity.

# Define Wrapper Function

In [45]:
def wrapper(annual_plan_df, num_plans=5, months=None):
    res = {}
    if months is None:
        months = annual_plan_df['month'].values
    df = annual_plan_df.query('month.isin(@months)')
    for idx, row in df.head(3).iterrows(): # first iterate through the months in the table
        month = row["month"]
        theme = row["theme_to_explore"]
        print(f"# {month}")
        markdown_lst = []
        markdown_lst.append(f"# {month}")
        markdown_lst.append(f"## Theme: {theme}")
        res[month] = dict()
        subjects = df.columns[2:].values
        for subject in subjects:  # for each month, iterate through the academic subjects
            pretty_subject = subject.replace("_", " ").title()
            markdown_lst.append(f"\n# {pretty_subject}")
            print(f"- {pretty_subject}")
            topic = row[subject]
            activity_ideas_parsed = prompt_wrapper(  # Generate activity ideas for each month-subject combo
                prompt=activity_idea_prompt, 
                parser=activity_ideas_parser,
                model=model,
                num_ideas=num_plans,
                month=month,
                theme=theme,
                subject=subject, 
                topic=topic
            )
            print("Generated ideas")
            res[month][subject] = activity_ideas_parsed.dict()
            for idea in res[month][subject]["ideas"]: #  Generate lesson plans for each idea
                activity_name = idea["name"]
                activity_description = idea["description"]
                lesson_plan = prompt_wrapper(
                    prompt=lesson_plan_prompt, 
                    parser=lesson_plan_parser,
                    model=model,
                    activity_name=activity_name, 
                    activity_description=activity_description, 
                    subject=subject
                )
                idea.update(lesson_plan)
                lesson_plan_markdown = convert_lesson_plan_to_markdown(idea)
                markdown_lst.append(f"\n{lesson_plan_markdown}")
                markdown_string = "\n".join(markdown_lst)
            print("Generated lesson plans")
                
        # Write the output of each month to a Markdown file
        output_file_path = f'../output/{idx + 1} {month} Lesson Plans.md'
        with open(output_file_path, "w") as output_file:
            output_file.write(markdown_string)

        print(f"Markdown output written to '{output_file_path}'.")
        print("*" * 50, "\n")
        # break
    
    # Write full results to a JSON file
    full_result_output_path = "../output/lesson_plan.json"
    with open(full_result_output_path , "w") as json_file:
        json.dump(res, json_file)
    print(f"JSON output written to '{full_result_output_path }'.")
    return res

# Run

In [46]:
annual_plan_df.head()

,month,theme_to_explore,language_arts,math,social_studies,science
0,August,Getting to Know One Another; Class Community,All About Me: Personal Narratives,Friendship Math: Counting and Sorting,Our Classroom Community,Exploring the school environment and local eco...
1,September,"On the Map, where we’re from",My Place on the Map: Descriptive Writing,Mapping Our Neighborhood; Measurement (units o...,My Hometown and Its History; Local History; ge...,"Water cycle, rivers; Geology, caves"
2,October,Fall; Halloween; Dia de los Muertos,Spooky Stories: Creative Writing,Place value; Estimation and graphs,"Fall celebrations (Halloween, Dia de los Muertos)",Animal science: Investigating Local Food Chains
3,November,Gratitude; Native Americans,Thankful Thoughts; Native American Storytelling,Addition and Subtraction,Native American History,Investigating Seasonal Changes and Plants
4,December,Winter Solstice and winter holidays,Holiday Traditions Around the World,Snowflake Symmetry; geometry (2-D shapes),Holidays Worldwide,Exploring Winter Animal Adaptations


In [48]:
lesson_plans = wrapper(annual_plan_df, months=["November", "December"])

# November
- Language Arts
Generated ideas
Generated lesson plans
- Math
Generated ideas
Generated lesson plans
- Social Studies
Generated ideas
Attempt 1 failed. Retrying...
Generated lesson plans
- Science
Generated ideas
Generated lesson plans
Markdown output written to '../output/4 November Lesson Plans.md'.
************************************************** 

# December
- Language Arts
Generated ideas
Generated lesson plans
- Math
Generated ideas
Generated lesson plans
- Social Studies
Generated ideas
Attempt 1 failed. Retrying...
Generated lesson plans
- Science
Generated ideas
Generated lesson plans
Markdown output written to '../output/5 December Lesson Plans.md'.
************************************************** 

JSON output written to '../output/lesson_plan.json'.


# View Results

In [41]:
display(JSON(lesson_plans))

<IPython.core.display.JSON object>